In [40]:
from google.colab import drive 

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [41]:
#initialize
import numpy as np
import pandas as pd

#load train_data from csv

csv_dataset ='/content/gdrive/MyDrive/AIC/movie_train.csv'

cols_dataset = list(pd.read_csv(csv_dataset, nrows =1))

csv_features = pd.read_csv(csv_dataset, index_col = 0, usecols =[i for i in cols_dataset if i != 'rating'])
csv_label = pd.read_csv(csv_dataset, index_col = 0, usecols=[0, 3])
csv_user = pd.read_csv(csv_dataset, index_col = 0, usecols=[0, 1])
csv_movie = pd.read_csv(csv_dataset, index_col = 0, usecols=[0, 2])

data_label = csv_label.values
data_label = data_label.reshape(-1, len(data_label))

data_user = csv_user.values
data_user = data_user.reshape(-1, len(data_user))

data_movie = csv_movie.values
data_movie = data_movie.reshape(-1, len(data_movie))

In [42]:
#display training data
matrix_R = []
R = []

max_movie = data_movie.max()
max_user = data_user.max()

for i in range(max_user):
    for j in range(max_movie):
        R.append(0)

    matrix_R.append(R)
    R = []

for i in range(len(data_user[0])):
    matrix_R[data_user[0][i]-1][data_movie[0][i]-1] = data_label[0][i]
    

In [43]:
#Baseline predictor
matrix_A = []
A = []

max_movie = data_movie.max()
max_user = data_user.max()
average_predictor = np.sum(data_label)/len(data_label[0])
for i in range(len(data_user[0])):
    for j in range(max_user+max_movie):
        A.append(0)

    matrix_A.append(A)
    A = []

for i in range(len(data_user[0])):
    matrix_A[i][data_user[0][i]-1] = 1
    matrix_A[i][data_movie[0][i]-1+max_user] = 1
    
trans_matrix_A = np.transpose(matrix_A)

pinv_matrix_A = np.linalg.pinv(matrix_A)

matrix_C = []
C = []

#prediction of bias
for i in range(len(data_user[0])):
    C.append(0)
    matrix_C.append(C)
    C = []
for i in range(len(data_user[0])):
    matrix_C[i][0] = data_label[0][i]-average_predictor

matrix_B = np.matmul(pinv_matrix_A, matrix_C)

In [44]:
#Rating matrix estimated by the baseline predictor
matrix_R_hat = []
R_hat = []
R_index = 0

max_movie = data_movie.max()
max_user = data_user.max()

for i in range(max_user):
    for j in range(max_movie):
        R_hat.append(0)
    matrix_R_hat.append(R_hat)
    R_hat = []

for i in range(max_user):
    for j in range(max_movie):
        matrix_R_hat[i][j] = round(matrix_B[i][0]+matrix_B[max_user+j][0]+average_predictor,2)

In [45]:
matrix_P = []
P = []
for i in range(max_user):
    for j in range(max_movie):
        P.append(0)
    matrix_P.append(P)
    P = []

for i in range(max_user):
    for j in range(max_movie):
        if matrix_R[i][j] != 0:
            matrix_P[i][j] = round(matrix_R[i][j] - matrix_R_hat[i][j],2)

In [46]:
#Similarity matrix
matrix_S = []
S = []
for i in range(max_movie):
    for j in range(max_movie):
        S.append(0)
    matrix_S.append(S)
    S = []

for i in range(max_movie):
    for j in range(max_movie):
        K = []
        upper_distance = 0
        lower_distance = 0
        lower_distance1 = 0
        lower_distance2 = 0
        distance = 0
        for k in range(max_user):
            if (matrix_R[k][i] != 0) & (matrix_R[k][j] != 0):
                K.append(k)
        for x in range(len(K)):
            upper_distance = upper_distance + (matrix_P[K[x]][i]*matrix_P[K[x]][j])
            lower_distance1 = lower_distance1 + (matrix_P[K[x]][i])**2
            lower_distance2 = lower_distance2 + (matrix_P[K[x]][j])**2
            lower_distance = (lower_distance1 * lower_distance2)**0.5
        if (lower_distance != 0):
            distance = upper_distance/lower_distance
        if (i != j):
            matrix_S[i][j] = round(distance,2)

In [47]:
#Rating matrix estimated by the Neighborhood Predictor
matrix_R_neigh = []
R_neigh = []

max_movie = data_movie.max()
max_user = data_user.max()

for i in range(max_user):
    for j in range(max_movie):
        R_neigh.append(0)
    matrix_R_neigh.append(R_neigh)
    R_neigh = []


for i in range(max_user):
    for j in range(max_movie):
        matrix_R_neigh[i][j] = round(matrix_B[i][0]+matrix_B[max_user+j][0]+average_predictor,2)

for k in range(max_user):
    for i in range(max_movie):
        upper_sum = 0
        lower_sum = 0
        for j in range(max_movie):
            if (abs(matrix_S[i][j]) > 0.1) & (matrix_P[k][j] != 0):
                upper_sum = upper_sum + matrix_S[i][j]*matrix_P[k][j]
                lower_sum = lower_sum + abs(matrix_S[i][j])
        if (lower_sum != 0):
            matrix_R_neigh[k][i] = round(matrix_R_hat[k][i] + (upper_sum/lower_sum),2)

# **Train Accuracy**

In [10]:
#train_accuracy by baseline predictor
correct = 0
sample = 0
for i in range(max_user):
    for j in range(max_movie):
        if matrix_R[i][j] != 0:
            sample = sample +1
            if (matrix_R[i][j] - round(matrix_R_hat[i][j])) == 0:
                correct = correct +1

accuracy = correct/sample*100

print("Accuracy on the {} samples : {:.2f}".format(sample, accuracy))

Accuracy on the 5216 samples : 43.35


In [11]:
#train_accuracy by neighborhood predictor
correct = 0
sample = 0
for i in range(max_user):
    for j in range(max_movie):
        if matrix_R[i][j] != 0:
            sample = sample +1
            if (matrix_R[i][j] - round(matrix_R_neigh[i][j])) == 0:
                correct = correct +1

accuracy = correct/sample*100

print("Accuracy on the {} samples : {:.2f}%".format(sample, accuracy))

Accuracy on the 5216 samples : 69.13%


In [48]:
#load test_data from csv

csv_testset ='/content/gdrive/MyDrive/AIC/movie_test.csv'
csv_testset_label ='/content/gdrive/MyDrive/AIC/movie_test_label.csv'

cols_testset = list(pd.read_csv(csv_testset, nrows =1))
cols_testset_label = list(pd.read_csv(csv_testset_label, nrows =1))

csv_label = pd.read_csv(csv_testset_label, index_col = 0, usecols=[0, 3])
csv_user = pd.read_csv(csv_testset, index_col = 0, usecols=[0, 1])
csv_movie = pd.read_csv(csv_testset, index_col = 0, usecols=[0, 2])

test_label = csv_label.values
test_label = test_label.reshape(-1, len(test_label))

test_user = csv_user.values
test_user = test_user.reshape(-1, len(test_user))

test_movie = csv_movie.values
test_movie = test_movie.reshape(-1, len(test_movie))

In [49]:
#display test data
matrix_R = []
R = []

max_movie = test_movie.max()
max_user = test_user.max()

for i in range(max_user):
    for j in range(max_movie):
        R.append(0)

    matrix_R.append(R)
    R = []

for i in range(len(test_user[0])):
    matrix_R[test_user[0][i]-1][test_movie[0][i]-1] = test_label[0][i]
    

In [20]:
#Rating matrix estimated by the baseline predictor
matrix_R_hat = []
R_hat = []
R_index = 0

max_movie = test_movie.max()
max_user = test_user.max()

for i in range(max_user):
    for j in range(max_movie):
        R_hat.append(0)
    matrix_R_hat.append(R_hat)
    R_hat = []

for i in range(max_user):
    for j in range(max_movie):
        matrix_R_hat[i][j] = round(matrix_B[i][0]+matrix_B[max_user+j][0]+average_predictor,2)

In [21]:
matrix_P = []
P = []
for i in range(max_user):
    for j in range(max_movie):
        P.append(0)
    matrix_P.append(P)
    P = []

for i in range(max_user):
    for j in range(max_movie):
        if matrix_R[i][j] != 0:
            matrix_P[i][j] = round(matrix_R[i][j] - matrix_R_hat[i][j],2)

In [22]:
#Similarity matrix
matrix_S = []
S = []
for i in range(max_movie):
    for j in range(max_movie):
        S.append(0)
    matrix_S.append(S)
    S = []

for i in range(max_movie):
    for j in range(max_movie):
        K = []
        upper_distance = 0
        lower_distance = 0
        lower_distance1 = 0
        lower_distance2 = 0
        distance = 0
        for k in range(max_user):
            if (matrix_R[k][i] != 0) & (matrix_R[k][j] != 0):
                K.append(k)
        for x in range(len(K)):
            upper_distance = upper_distance + (matrix_P[K[x]][i]*matrix_P[K[x]][j])
            lower_distance1 = lower_distance1 + (matrix_P[K[x]][i])**2
            lower_distance2 = lower_distance2 + (matrix_P[K[x]][j])**2
            lower_distance = (lower_distance1 * lower_distance2)**0.5
        if (lower_distance != 0):
            distance = upper_distance/lower_distance
        if (i != j):
            matrix_S[i][j] = round(distance,2)

In [ ]:
#Rating matrix estimated by the Neighborhood Predictor
matrix_R_neigh = []
R_neigh = []

max_movie = data_movie.max()
max_user = data_user.max()

for i in range(max_user):
    for j in range(max_movie):
        R_neigh.append(0)
    matrix_R_neigh.append(R_neigh)
    R_neigh = []


for i in range(max_user):
    for j in range(max_movie):
        matrix_R_neigh[i][j] = round(matrix_B[i][0]+matrix_B[max_user+j][0]+average_predictor,2)

for k in range(max_user):
    for i in range(max_movie):
        upper_sum = 0
        lower_sum = 0
        for j in range(max_movie):
            if (abs(matrix_S[i][j]) > 0.6) & (matrix_P[k][j] != 0):
                upper_sum = upper_sum + matrix_S[i][j]*matrix_P[k][j]
                lower_sum = lower_sum + abs(matrix_S[i][j])
        if (lower_sum != 0):
            matrix_R_neigh[k][i] = round(matrix_R_hat[k][i] + (upper_sum/lower_sum),2)

# **Test accuracy**


In [17]:
#test_accuracy by baseline predictor
correct = 0
sample = 0
for i in range(max_user):
    for j in range(max_movie):
        if matrix_R[i][j] != 0:
            sample = sample +1
            if (matrix_R[i][j] - round(matrix_R_hat[i][j])) == 0:
                correct = correct +1

accuracy = correct/sample*100

print("Accuracy on the {} samples : {:.2f}%".format(sample, accuracy))

Accuracy on the 1304 samples : 36.20%


In [ ]:
#test_accuracy by neighborhood predictor tolerance = 0.4
correct = 0
sample = 0
for i in range(max_user):
    for j in range(max_movie):
        if matrix_R[i][j] != 0:
            sample = sample +1
            if (matrix_R[i][j] - round(matrix_R_neigh[i][j])) == 0:
                correct = correct +1

accuracy = correct/sample*100

print("Accuracy on the {} samples : {:.2f}%".format(sample, accuracy))

Accuracy on the 1304 samples : 54.37%


In [26]:
#Rating matrix estimated by the Neighborhood Predictor
matrix_R_neigh = []
R_neigh = []

max_movie = data_movie.max()
max_user = data_user.max()

for i in range(max_user):
    for j in range(max_movie):
        R_neigh.append(0)
    matrix_R_neigh.append(R_neigh)
    R_neigh = []


for i in range(max_user):
    for j in range(max_movie):
        matrix_R_neigh[i][j] = round(matrix_B[i][0]+matrix_B[max_user+j][0]+average_predictor,2)

for k in range(max_user):
    for i in range(max_movie):
        upper_sum = 0
        lower_sum = 0
        for j in range(max_movie):
            if (abs(matrix_S[i][j]) > 0.3) & (matrix_P[k][j] != 0):
                upper_sum = upper_sum + matrix_S[i][j]*matrix_P[k][j]
                lower_sum = lower_sum + abs(matrix_S[i][j])
        if (lower_sum != 0):
            matrix_R_neigh[k][i] = round(matrix_R_hat[k][i] + (upper_sum/lower_sum),2)

In [27]:
#test_accuracy by neighborhood predictor tolerance = 0.7
correct = 0
sample = 0
for i in range(max_user):
    for j in range(max_movie):
        if matrix_R[i][j] != 0:
            sample = sample +1
            if (matrix_R[i][j] - round(matrix_R_neigh[i][j])) == 0:
                correct = correct +1

accuracy = correct/sample*100

print("Accuracy on the {} samples : {:.2f}%".format(sample, accuracy))

Accuracy on the 1304 samples : 55.14%


In [30]:
#Rating matrix estimated by the Neighborhood Predictor
matrix_R_neigh = []
R_neigh = []

max_movie = data_movie.max()
max_user = data_user.max()

for i in range(max_user):
    for j in range(max_movie):
        R_neigh.append(0)
    matrix_R_neigh.append(R_neigh)
    R_neigh = []


for i in range(max_user):
    for j in range(max_movie):
        matrix_R_neigh[i][j] = round(matrix_B[i][0]+matrix_B[max_user+j][0]+average_predictor,2)

for k in range(max_user):
    for i in range(max_movie):
        upper_sum = 0
        lower_sum = 0
        for j in range(max_movie):
            if (abs(matrix_S[i][j]) != 0) & (matrix_P[k][j] != 0):
                upper_sum = upper_sum + matrix_S[i][j]*matrix_P[k][j]
                lower_sum = lower_sum + abs(matrix_S[i][j])
        if (lower_sum != 0):
            matrix_R_neigh[k][i] = round(matrix_R_hat[k][i] + (upper_sum/lower_sum),2)

In [31]:
#test_accuracy by neighborhood predictor tolerance = 1
correct = 0
sample = 0
for i in range(max_user):
    for j in range(max_movie):
        if matrix_R[i][j] != 0:
            sample = sample +1
            if (matrix_R[i][j] - round(matrix_R_neigh[i][j])) == 0:
                correct = correct +1

accuracy = correct/sample*100

print("Accuracy on the {} samples : {:.2f}%".format(sample, accuracy))

Accuracy on the 1304 samples : 55.21%
